# Natural Language Processing with Tensorflow

This notebook contains:
  * Downloading a text dataset
  * Visualizing the data
  * Converting text into numbers using tokenization
  * Turning tokenized text into an embedding
  * Modelling a text data
    * Starting with a baseline (TF-IDF)
    * Building several deep learning text models
      * Dense, LSTM, GRU, Conv1D, Transfer Learning
  * Comparing the performance of each of our models
  * Combining our models into an ensemble
  * Saving and loading a trained model
  * Finding the most wrong predictions

## Checking the GPU

In [68]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-98b3ee16-089c-361f-d34a-aa8ff008dce5)


## Get helper functions

In [69]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-02-11 15:12:01--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-02-11 15:12:01 (65.2 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [70]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

# Download a text dataset

In [71]:
# Download data
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

--2023-02-11 15:12:01--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.16.128, 172.253.62.128, 172.253.115.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.16.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2023-02-11 15:12:01 (107 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [72]:
# Unzip data
unzip_data("nlp_getting_started.zip")

# Visualizing the dataset

In [73]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [74]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac = 1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [75]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [76]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [77]:
# How many samples total
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df)+len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [78]:
# Visualizing some random samples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text: {text}")
  print("---")

Target: 0 (not real disaster)
Text: I added a video to a @YouTube playlist http://t.co/612BsbVw8K siren 1 gameplay/walkthrough part 1
---
Target: 0 (not real disaster)
Text: I can't believe @myfriendmina photo bombed a screenshot
---
Target: 1 (real disaster)
Text: 1) 'Investigators say a Virgin Galactic spaceship crash was caused by structural failure after the co-pilot unlocked a braking system early'
---
Target: 1 (real disaster)
Text: @peterjukes A crime that killed/displaced millions. In which systematic torture was carried out. But look. GrahamWP fired a gun! Arrest him!
---
Target: 0 (not real disaster)
Text: aggressif is so bloody aggressive
---


### Split the data into training and validation sets

In [79]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_df_shuffled['text'].to_numpy(), train_df_shuffled['target'].to_numpy(), test_size=0.1, random_state=42)

In [80]:
# Check the sizes of train and val sets
len(X_train), len(X_val), len(y_train), len(y_val)

(6851, 762, 6851, 762)

# Converting Text into Numbers

### Text Vectorization

In [81]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None)

In [82]:
# Find average number of tokens in training Tweets
round(sum([len(i.split()) for i in X_train])/len(X_train))

15

In [83]:
#@title Default title text
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                   output_mode="int",
                                   output_sequence_length=max_length)

In [84]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(X_train)

In [85]:
# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [86]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(X_train)
print(f"Original text:\n{random_sentence}\nVectorized text: \n{text_vectorizer(random_sentence)}")

Original text:
Hibernating pbx irrespective of pitch fatality careerism pan: crbZFZ
Vectorized text: 
[   1 9891    1    6 2802  626    1 4962    1    0    0    0    0    0
    0]


In [87]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

In [88]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length="max_length",
                             name="embedding_1")

embedding

In [89]:
# Get a random sentence from training set
random_sentence = random.choice(X_train)
print(f"Original text:\n{random_sentence}\nEmbedded version:")
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
@drvox Trump can say nice things about nuclear power but 'drill baby drill!' + AGW denial = policy disaster for nuclear power.
Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02891598,  0.031052  , -0.04259444, ...,  0.00355012,
         -0.03083894, -0.00286206],
        [-0.02084574,  0.04024902,  0.0328524 , ...,  0.00364748,
         -0.00832777,  0.01227106],
        [ 0.00998317, -0.00046135,  0.0075789 , ...,  0.04629404,
          0.02903905,  0.03523573],
        ...,
        [ 0.04563827, -0.00558738,  0.02728764, ...,  0.03995012,
         -0.04200175, -0.04344925],
        [-0.02891598,  0.031052  , -0.04259444, ...,  0.00355012,
         -0.03083894, -0.00286206],
        [ 0.02218534, -0.04892312,  0.0205355 , ...,  0.04763149,
          0.03854365, -0.00382311]]], dtype=float32)>

In [90]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.02891598,  0.031052  , -0.04259444,  0.04668799,  0.04648152,
        0.03688559, -0.03083288,  0.0472468 ,  0.04423917,  0.01431871,
        0.04574499,  0.00101992, -0.03348169, -0.04024922, -0.01159779,
        0.00796794,  0.0371669 , -0.01726158, -0.0047613 , -0.03242297,
       -0.04860789,  0.04674042, -0.04386712, -0.01002468, -0.01178382,
        0.04251326, -0.02384456, -0.03695989,  0.00729775,  0.00154523,
       -0.00938331, -0.04150695,  0.04369299,  0.0086431 , -0.02134447,
       -0.01227504,  0.03365128,  0.01810024, -0.04331626,  0.00401994,
       -0.0496999 ,  0.03181748, -0.01340009, -0.03492383,  0.04319036,
        0.04389813, -0.01653969,  0.00823904,  0.03364858, -0.00253012,
        0.00917755,  0.02465152,  0.02712983,  0.00038294, -0.03039302,
       -0.03657744, -0.0382153 , -0.04216867, -0.04178201, -0.00010787,
       -0.02791209,  0.03938676, -0.04220512, -0.04174829,  0.02108966,
       -0.034285

# Modelling a text dataset

* **Model 0**: Naive bayes (baseline)
* **Model 1**: Feed-forward neural network (dense model)
* **Model 2**: LSTM model
* **Model 3**: GRU model
* **Model 4**: Bidirectional-LSTM model
* **Model 5**: 1D Convolutional Neural Network
* **Model 6**: Tensorflow Hub Pretrained Feature Extractor
* **Model 7**: Same as model 6 with 10% of training data

### Model 0: Getting a baseline

In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data
model_0.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [92]:
baseline_score = model_0.score(X_val, y_val)
print(f"Our baseline model achieves the accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves the accuracy of: 79.27%


In [93]:
# Make predictions
baseline_preds = model_0.predict(X_val)
baseline_preds[:20]


array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments
  * Accuracy
  * Precision
  * Recall
  * F1-Score

In [94]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1":model_f1}
  return model_results

In [95]:
# Get baseline results
baseline_results = calculate_results(y_true=y_val, y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A Simple dense model

In [96]:
# Create tensorboard callback
from helper_functions import create_tensorboard_callback

# Create directory to save tensorboard logs
SAVE_DIR = "model_logs"

In [97]:
# Build model with the functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [98]:
# Compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [99]:
# Get summary of the model
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [100]:
# Fit the model
model_1.history = model_1.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=[X_val, y_val],
                              callbacks=[create_tensorboard_callback(dir_name="SAVE_DIR",
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: SAVE_DIR/simple_dense_model/20230211-151203
Epoch 1/5
215/215 [==============================] - 9s 36ms/step - loss: 0.6109 - accuracy: 0.6914 - val_loss: 0.5378 - val_accuracy: 0.7546
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.4423 - accuracy: 0.8194 - val_loss: 0.4697 - val_accuracy: 0.7900
Epoch 3/5
215/215 [==============================] - 4s 20ms/step - loss: 0.3472 - accuracy: 0.8608 - val_loss: 0.4590 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2857 - accuracy: 0.8914 - val_loss: 0.4637 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2388 - accuracy: 0.9121 - val_loss: 0.4760 - val_accuracy: 0.7861


In [101]:
# Check the results
model_1.evaluate(X_val, y_val)

24/24 [==============================] - 0s 5ms/step - loss: 0.4760 - accuracy: 0.7861


[0.4759683310985565, 0.7860892415046692]

In [102]:
embedding.weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[ 0.02005848, -0.03127361, -0.05478824, ...,  0.00946877,
         -0.00860927, -0.01672559],
        [-0.03381377,  0.03605807, -0.04709477, ...,  0.00955085,
         -0.03546685, -0.00589119],
        [ 0.01106572,  0.02052376, -0.06031356, ...,  0.0316435 ,
         -0.03630352, -0.04603537],
        ...,
        [ 0.01476702,  0.02115197,  0.01908422, ..., -0.01618927,
         -0.0165259 ,  0.02588346],
        [-0.08514438,  0.06772456, -0.04214695, ...,  0.04103063,
         -0.00958938, -0.02522461],
        [-0.02207695,  0.02315003, -0.02047201, ...,  0.02989179,
         -0.08823354, -0.09251369]], dtype=float32)>]

In [103]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [104]:
# Make predictions
model_1_pred_probs = model_1.predict(X_val)
model_1_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.41278526],
       [0.7447355 ],
       [0.99775946],
       [0.10781892],
       [0.11343618],
       [0.9377237 ],
       [0.9122917 ],
       [0.99273235],
       [0.9701263 ],
       [0.2609719 ]], dtype=float32)

In [105]:
# Turn prediction probabilities into single dimension tensor of floats
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [106]:
# Calculate model_1 metrics
model_1_results = calculate_results(y_true = y_val, y_pred = model_1_preds)
model_1_results

{'accuracy': 78.60892388451444,
 'precision': 0.7903277546022673,
 'recall': 0.7860892388451444,
 'f1': 0.7832971347503846}

### Model 2: LSTM

In [107]:
# Set random seed and create embedding layer
tf.random.set_seed(42)
from tensorflow.keras import layers
model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")

# Create LSTM model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)
print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
# x = layers.Dense(1, activation="sigmoid")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)


In [108]:
# Compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [109]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [110]:
# Fit the model
model_2_history = model_2.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=(X_val, y_val),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20230211-151246
Epoch 1/5
215/215 [==============================] - 20s 75ms/step - loss: 0.5108 - accuracy: 0.7449 - val_loss: 0.4605 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 4s 19ms/step - loss: 0.3180 - accuracy: 0.8713 - val_loss: 0.5137 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.2199 - accuracy: 0.9165 - val_loss: 0.6004 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.1563 - accuracy: 0.9456 - val_loss: 0.6340 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1100 - accuracy: 0.9590 - val_loss: 0.8822 - val_accuracy: 0.7598


In [111]:
# Make prediction on the validation dataset
model_2_pred_probs = model_2.predict(X_val)
model_2_pred_probs.shape, model_2_pred_probs[:10]

24/24 [==============================] - 1s 7ms/step


((762, 1), array([[0.02066   ],
        [0.7351706 ],
        [0.9991855 ],
        [0.05974191],
        [0.00668198],
        [0.9993137 ],
        [0.81485784],
        [0.99949455],
        [0.99945444],
        [0.22054256]], dtype=float32))

In [112]:
# Round up predictions and reduce it to 1D array
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [113]:
# Calculate LSTM model results
model_2_results = calculate_results(y_val, model_2_preds)
model_2_results

{'accuracy': 75.98425196850394,
 'precision': 0.7599947657549518,
 'recall': 0.7598425196850394,
 'f1': 0.7584680708642795}

### Model 3: GRU

In [114]:
# Set random seed and create embedding layer
tf.random.set_seed(42)
from tensorflow.keras import layers
model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="GRU")

In [115]:
# Compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [116]:
model_3.summary()

Model: "GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru_1 (GRU)                 (None, 64)                37248     
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________________

In [117]:
# Fit the model
model_3_history = model_3.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=(X_val, y_val),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"GRU")])

Saving TensorBoard log files to: model_logs/GRU/20230211-151321
Epoch 1/5
215/215 [==============================] - 14s 49ms/step - loss: 0.5264 - accuracy: 0.7266 - val_loss: 0.4565 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3183 - accuracy: 0.8670 - val_loss: 0.4874 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2161 - accuracy: 0.9175 - val_loss: 0.5701 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1521 - accuracy: 0.9473 - val_loss: 0.6262 - val_accuracy: 0.7822
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1158 - accuracy: 0.9597 - val_loss: 0.6088 - val_accuracy: 0.7730


In [118]:
# Make prediction on validation data
model_3_pred_probs = model_3.predict(X_val)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[0.3566097 ],
       [0.9245423 ],
       [0.9967488 ],
       [0.1779081 ],
       [0.01083807],
       [0.9907266 ],
       [0.8044865 ],
       [0.9964379 ],
       [0.9963266 ],
       [0.32198906]], dtype=float32)

In [119]:
# Round up the prediction and convert it to 1D array
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [120]:
# Calculate result
model_3_results = calculate_results(y_val, model_3_preds)
model_3_results

{'accuracy': 77.29658792650919,
 'precision': 0.7727692945264432,
 'recall': 0.7729658792650919,
 'f1': 0.7721544244341451}

### Model 4: Bidirectional RNN Model

In [121]:
# Set random seed and create embedding layer
tf.random.set_seed(42)
from tensorflow.keras import layers
model_4_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_4")

# Build a Bidirectional RNN in tensorflow
input = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(input)
x = model_4_embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
output = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(input, output, name="model_4_Bidirectional")

In [122]:
# Compile
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [123]:
# Get a summary of our bidirectional model
model_4.summary()

Model: "model_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_4 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [124]:
# Fit the model
model_4_history = model_4.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=(X_val, y_val),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "bidirectional_RNN")])

Saving TensorBoard log files to: model_logs/bidirectional_RNN/20230211-151346
Epoch 1/5
215/215 [==============================] - 18s 57ms/step - loss: 0.5090 - accuracy: 0.7469 - val_loss: 0.4564 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 5s 21ms/step - loss: 0.3136 - accuracy: 0.8724 - val_loss: 0.5235 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2122 - accuracy: 0.9188 - val_loss: 0.5607 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 4s 20ms/step - loss: 0.1479 - accuracy: 0.9496 - val_loss: 0.6463 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 4s 20ms/step - loss: 0.1050 - accuracy: 0.9639 - val_loss: 0.6791 - val_accuracy: 0.7638


In [125]:
# Make predictions with bidirectional RNN on validation data
model_4_pred_probs = model_4.predict(X_val)
model_4_pred_probs[:10]

24/24 [==============================] - 3s 6ms/step


array([[0.04763677],
       [0.8567207 ],
       [0.9991248 ],
       [0.18376713],
       [0.00585338],
       [0.99671894],
       [0.92211366],
       [0.9994355 ],
       [0.9996439 ],
       [0.21316446]], dtype=float32)

In [126]:
# Convert prediction probabilities to labels
model_4_pred = tf.squeeze(tf.round(model_4_pred_probs))
model_4_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [127]:
# Calculate bidirection model results
model_4_results = calculate_results(y_val, model_4_pred)
model_4_results

{'accuracy': 76.37795275590551,
 'precision': 0.76369847151459,
 'recall': 0.7637795275590551,
 'f1': 0.7626922180365151}

### Model 5: Conv1D (Convolutional Neural Network for Text)

In [128]:
# Test out the embedding, 1D Convolutional and max pooling
embedding_test = embedding(text_vectorizer(['This is a test sentence']))
conv_1d = layers.Conv1D(filters=32, kernel_size=5, activation="relu")
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [129]:
embedding_test[:1], max_pool_output[:1], conv_1d_output[:1]

(<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
 array([[[ 0.04363716,  0.01619624, -0.02075452, ..., -0.02501463,
          -0.03859189,  0.00695223],
         [-0.07697   ,  0.04064272,  0.01288766, ...,  0.00499057,
          -0.02093619, -0.06619073],
         [-0.05679797,  0.03346261, -0.00683506, ..., -0.02128714,
           0.00986681,  0.0318494 ],
         ...,
         [ 0.02005848, -0.03127361, -0.05478824, ...,  0.00946877,
          -0.00860927, -0.01672559],
         [ 0.02005848, -0.03127361, -0.05478824, ...,  0.00946877,
          -0.00860927, -0.01672559],
         [ 0.02005848, -0.03127361, -0.05478824, ...,  0.00946877,
          -0.00860927, -0.01672559]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 32), dtype=float32, numpy=
 array([[0.03795921, 0.11093213, 0.05089401, 0.09841204, 0.07477862,
         0.13296884, 0.        , 0.02668877, 0.16576697, 0.01472138,
         0.02465613, 0.09620833, 0.07230133, 0.10253595, 0.04341869,
         0.03644947, 0.0429785 

In [131]:
# Set random seed and create embedding layer
tf.random.set_seed(42)
from tensorflow.keras import layers
model_5_embedding = layers.Embedding(input_dim=max_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_5")
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_5_embedding(x)
x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

In [132]:
# model_5 summary
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 15, 128)           1920      
                                                                 
 conv1d_4 (Conv1D)           (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d_4 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_8 (Dense)             (None, 1)              

In [134]:
# Compile model_5
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [135]:
# Fit the model
model_5_history = model_5.fit(X_train, y_train, epochs=5,
                              validation_data=(X_val, y_val),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20230211-153018
Epoch 1/5
215/215 [==============================] - 17s 67ms/step - loss: 0.6401 - accuracy: 0.6227 - val_loss: 0.6221 - val_accuracy: 0.6365
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.6121 - accuracy: 0.6583 - val_loss: 0.6110 - val_accuracy: 0.6535
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.6021 - accuracy: 0.6672 - val_loss: 0.6115 - val_accuracy: 0.6509
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.5965 - accuracy: 0.6732 - val_loss: 0.6134 - val_accuracy: 0.6549
Epoch 5/5
215/215 [==============================] - 5s 21ms/step - loss: 0.5903 - accuracy: 0.6795 - val_loss: 0.6166 - val_accuracy: 0.6391


In [136]:
# Make predictions with model_5
model_5_pred_probs = model_5.predict(X_val)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 4ms/step


array([[0.43149304],
       [0.703212  ],
       [0.7490768 ],
       [0.24598719],
       [0.32556328],
       [0.74265844],
       [0.7275839 ],
       [0.47240546],
       [0.58903086],
       [0.3485651 ]], dtype=float32)

In [137]:
# Convert predictions to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 0., 1., 0.], dtype=float32)>

In [138]:
# Calculate model_5 results
model_5_results = calculate_results(y_val, model_5_preds)
model_5_results

{'accuracy': 63.91076115485564,
 'precision': 0.6461011254918971,
 'recall': 0.6391076115485564,
 'f1': 0.6219187074195016}